In [279]:
from glob import glob
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import plotly.graph_objs as go
from mooringParams import paramsDAFT1 as processParams # Each mooring has a specific params file with 
from scipy.interpolate import splprep,splev

In [11]:
dfs = []
for file in tqdm(glob(processParams.evParams.outputDir+'exports\\'+'*(regions).csv')):
    dfs.append(pd.read_csv(file))
dfR = pd.concat(dfs)
dfs = []

for file in tqdm(glob(processParams.evParams.outputDir+'exports\\'+'*(targets).csv')):
    dfs.append(pd.read_csv(file))
dfT = pd.concat(dfs)
cols = np.append(dfT.columns[0:18],['Ping_number']).tolist()
dfT = dfT[cols].drop(columns=['Region_name', 'Region_class'])
dfR = dfR.drop(columns=['Region_name', 'Region_class','Target_length_mean'])
dfR = dfR[dfR.TS_max < -30]
dfT = dfT[dfT.Region_ID.isin(dfR.Region_ID)]

In [358]:
def targetSpline(df, processID, regionID,direct=False):
    df = df[(df.Region_ID==regionID) & (df.Process_ID==processID)]
    tck, u  = splprep([df.Distance_major_axis, df.Distance_minor_axis,df.Target_true_depth],k=3)
    if direct:
        x_knots, y_knots, z_knots = splev([0,1], tck)
    else:
        x_knots, y_knots, z_knots = splev(np.linspace(0,1,len(df.Distance_major_axis)), tck)
    return  x_knots, y_knots, z_knots

def trackPlot(df,processID, regionID, plotElements =['track_smoothed','track','track_direct','targets']):
    # plotElements: list of elements to include:
    # targets: target locations
    # track: line between targets
    # track_smoothed: Cubic spline with equal resolution to the number of targets
    # track_direct: "flux" line from start to end target based on spline    
    x_knots, y_knots, z_knots = targetSpline(df, processID, regionID ,direct=False)
    track_smoothed = go.Scatter3d(x=x_knots, y=y_knots, z=z_knots,mode='lines',name='Smoothed Line')
    
    x_knots, y_knots, z_knots = targetSpline(df, processID, regionID ,direct=True)
    track_direct = go.Scatter3d(x=x_knots, y=y_knots, z=z_knots,mode='lines',name='Direct Path')

    dfCur = df[(df.Region_ID==regionID) & (df.Process_ID==processID)]
    targets=go.Scatter3d(x=dfCur.Distance_major_axis, y=dfCur.Distance_minor_axis, z=dfCur.Target_true_depth,mode='markers',
        marker=dict(size=8,color='black', colorscale='plasma',opacity=0.8,colorbar=dict(title='TS',thickness=30),showscale=False),name='Targets')
    track = go.Scatter3d(x=dfCur.Distance_major_axis, y=dfCur.Distance_minor_axis, z=dfCur.Target_true_depth,mode='lines',name='Target Line')

    allElements = {'track_smoothed':track_smoothed,'track':track,'track_direct':track_direct,'targets':targets}
    fig = go.Figure(data=[allElements[i] for i in plotElements])
    split = (np.ceil(np.max([dfCur.Distance_major_axis.max() - dfCur.Distance_major_axis.min(),dfCur.Distance_minor_axis.max() - dfCur.Distance_minor_axis.min(),dfCur.Target_true_depth.max() - dfCur.Target_true_depth.min()]))/2)+.25
    fig.update_layout(showlegend=True,scene = dict(
                        xaxis_title='Distance_major_axis',
                        yaxis_title='Distance_minor_axis',
                        zaxis_title='Depth'),
                        #xaxis = dict(nticks=4, range=[np.median(dfCur.Distance_major_axis)-split, np.median(dfCur.Distance_major_axis)+split]),
                        #yaxis = dict(nticks=4, range=[np.median(dfCur.Distance_minor_axis)-split, np.median(dfCur.Distance_minor_axis)+split]),
                       # zaxis = dict(nticks=4, range=[np.median(dfCur.Target_true_depth)-split, np.median(dfCur.Target_true_depth)+split])),
                        width=700,
                        margin=dict(r=20, b=10, l=10, t=10))

    fig.show()

trackPlot(dfT,378, 21)